# Modélisations

## Création de la base d'entraînement et de celle de test

In [82]:
import requests
import zipfile
import io
import pandas as pd
from shutil import rmtree
from jyquickhelper import add_notebook_menu
from unidecode import unidecode
import numpy as np
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
add_notebook_menu()


In [2]:
insee = pd.read_csv('Données élections et INSEE.csv')
insee = insee.drop("Unnamed: 0", axis = 1)

C:\Users\Xavier\AppData\Local\Temp\ipykernel_16800\3911578995.py:1: DtypeWarning: Columns (1,1911) have mixed types. Specify dtype option on import or set low_memory=False.
  insee = pd.read_csv('Données élections et INSEE.csv')


In [4]:
# On crée la dataframe avec les observations où le maire s'est représenté
insee2 = insee[insee["Tentative de réélection"]==True]

In [5]:
# On crée le vecteur contenant résultat
Reelection = insee2["Nom de l'élu en 2020"] == insee2["Nom de l'élu en 2014"]

In [6]:
# On va réduire insee2 en ne gardant que les données de 2014 et 2020 qui sont des valeurs chiffrées
# Valeurs chiffrées
temp = insee2.select_dtypes(include = ["int64", "float64"])
insee2 = pd.concat([insee2["CODGEO"], temp], axis = 1)
# Années gardées
for i in insee2.columns :
    if i[0:3]!="P19" and i[0:3]!="P13" and i!="CODGEO":
        insee2 = insee2.drop(i, axis = 1)

Si l'on veut être surs que nos modèles soient comparables, il faut centrer et réduire nos données.

In [7]:
temp2 = insee["CODGEO"]

var = insee2.std()
moy = insee2.mean()

insee2 = (insee2-moy)/var
insee2["CODGEO"]=temp2

C:\Users\Xavier\AppData\Local\Temp\ipykernel_16800\544017825.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  var = insee2.std()
C:\Users\Xavier\AppData\Local\Temp\ipykernel_16800\544017825.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  moy = insee2.mean()


In [8]:
# Une partie de nos colonnes contiennent (énormément) de NaNs.
# Seulement quelques colonnes, redondantes entre 2013 et 2020, sont concernées.
# Par simplicité, nous les enlevons donc.
liste = []
for i in insee2.columns:
        if insee2[i].isnull().values.any()==True :
            liste.append(i)
            
insee2 = insee2.drop(liste, axis = 1)        

On crée maintenant les bases d'entraînement et de test pour nos différentes modélisations.

In [110]:
X_train, X_test, y_train, y_test = train_test_split(insee2, Reelection, test_size=0.2, random_state=18)

Nous allons tester plusieurs méthodes. Afin de savoir laquelle est la "meilleure", nous allons les comparer grâce à quatre indicateurs : l'exactitude (accuracy), la précision, le rappel (recall) ainsi que la statistique F1.

L'exactitude regarde à quel point les données simulées sont proches de la réalité et regarde donc la proportion des résultats dont l'issue est la bonne.

La précision correspond au nombre de vrais positifs sur le nombre de prédictions positives.

Le rappel correspond au nombre de vrais positifs sur le nombre réel de positifs.

La statistique F1 correspond à $\frac{2 \times precision \times rappel}{precision + rappel}$.

In [116]:
# Création de la base contenant ces données :
Stats_Modeles = pd.DataFrame(columns = ["Modèle", "Accuracy", "Precision", "Recall", "Score F1", "Coefficients tous nuls ?"])

# Ajoute une ligne à la dataframe
def plusUneLigne(M, y_t, y_p, C):
    
    # On évalue les stats du modèle
    ac = sklearn.metrics.accuracy_score(y_t, y_p)
    prec = sklearn.metrics.precision_score(y_t, y_p)
    recall = sklearn.metrics.recall_score(y_t, y_p)
    f1 = sklearn.metrics.f1_score(y_t, y_p)
    
    return Stats_Modeles.append({"Modèle" : M,
                                          "Accuracy" : ac,
                                          "Precision" : prec,
                                          "Recall" : recall,
                                          "Score F1" : f1,
                                          "Coefficients tous nuls ?" : C},
                                          ignore_index=True)

## 1 - Méthode du lasso

Nous avons ici une base de données avec beaucoup de variables. De ce fait, avec la méthode du lasso, qui ajoute une pénalisation correspondant à la norme 2 du vecteur des $\beta_i$. La formule du lasso est la suivante :
$ \hat{\beta} = argmin_{\beta \in \mathbb{R}^k } = \frac{1}{n} \sum_{i=1}^{n}{(Y_i - X_i^{'}  \beta)^2 + \lambda ||\beta||_1}$ où $\lambda \in \mathbb{R}^{+}$ est un coefficient de pondération. Plus $\lambda$ est grand, plus le nombre de coefficients $\beta_i$ égal à zéro augmente.

Ce modèle est continue donc on ne trouvera pas un $\hat{y}_i \in \{ 0; 1\}$ mais dans $[\ 0; 1]\ $ correspondant plutôt à une probabilité d'être réélu(e). On binarise alors notre vecteur en donnant la valeur de un aux probabilités supérieures à $\frac{1}{2}$ et zéro à celles inférieures : $ \hat{y_i}' = \mathbb{1}_{ [\ \frac{1}{2}; 1]\ }(y_i) $.

In [117]:
lambdas = [ 0.1, 0.001, 0.0001, 0.00001, 0]

for i in lambdas:
    # On établie la régression
    clf = linear_model.Lasso(alpha=i)
    clf.fit(X_train.drop("CODGEO", axis=1), y_train)
   
    y_pred = clf.predict(X_test.drop("CODGEO", axis=1))
    
    for j in range(len(y_pred)):
        if y_pred[j]<0.5:
            y_pred[j] = 0
        else :
            y_pred[j] = 1
    
    nom_temp = "Lasso (α = " + str(i)+ ")"
    nul = np.array_equal(clf.coef_, np.zeros(X_test.shape[1]-1))
    
    Stats_Modeles = plusUneLigne(nom_temp, y_test, y_pred, nul)

C:\Users\Xavier\AppData\Local\Temp\ipykernel_16800\4166890010.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return Stats_Modeles.append({"Modèle" : M,
C:\Users\Xavier\AppData\Local\Temp\ipykernel_16800\4166890010.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return Stats_Modeles.append({"Modèle" : M,
C:\Users\Xavier\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.076e+02, tolerance: 1.907e-01
  model = cd_fast.enet_coordinate_descent(
C:\Users\Xavier\AppData\Local\Temp\ipykernel_16800\4166890010.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas

## 2 - Régression logistique

Le second modèle que nous allons utiliser correspond au modèle logistique.

In [119]:
solvers = ["lbfgs", "liblinear", "sag" ]

for i in solvers:
    
    clf = LogisticRegression(random_state=0, solver = i)
    clf.fit(X_train.drop("CODGEO", axis=1), y_train)
    
    y_pred = clf.predict(X_test.drop("CODGEO", axis=1))
    
    nom_temp = "Logistique (solver = " + i + ")"
    nul = np.array_equal(clf.coef_, np.zeros(X_test.shape[1]-1))
    
    Stats_Modeles = plusUneLigne(nom_temp, y_test, y_pred, nul)

C:\Users\Xavier\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Xavier\AppData\Local\Temp\ipykernel_16800\4166890010.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return Stats_Modeles.append({"Modèle" : M,
C:\Users\Xavier\AppData\Local\Temp\ipykernel_16800\4166890010.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return Stats_Modeles.append({"Modè

## 3 - Méthode des k plus proches voisins

La méthode des k plus proches voisins ($ k \in \mathbb{N}^* $) regarde l'issue la plus présente parmi les k observations qui sont les plus proches de notre observation donc l'issue est à prévoir. La notion de proche peut varier suivant la distance choisie.

In [121]:
combien_voisins = [5, 11, 101, 1001]   

for i in combien_voisins :    
    clf = KNeighborsClassifier(n_neighbors=50)
    clf.fit(X_train.drop("CODGEO", axis=1), y_train)
    y_pred = clf.predict(X_test.drop("CODGEO", axis=1))

    nom_temp = str(i) + " plus proches voisins"

    Stats_Modeles = plusUneLigne(nom_temp, y_test, y_pred, np.nan)

C:\Users\Xavier\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Xavier\AppData\Local\Temp\ipykernel_16800\4166890010.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return Stats_Modeles.append({"Modèle" : M,
C:\Users\Xavier\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically pres

In [122]:
Stats_Modeles

,Modèle,Accuracy,Precision,Recall,Score F1,Coefficients tous nuls ?
0,Lasso (α = 0.1),0.842133,0.842133,1.000000,0.914302,True
1,Lasso (α = 0.001),0.842133,0.842133,1.000000,0.914302,False
2,Lasso (α = 0.0001),0.840257,0.841837,0.997772,0.913195,False
3,Lasso (α = 1e-05),0.839989,0.842346,0.996499,0.912961,False
4,Lasso (α = 0),0.839453,0.842261,0.995863,0.912644,False
5,Logistique (solver = lbfgs),0.838917,0.842361,0.994908,0.912301,False
6,Logistique (solver = liblinear),0.838649,0.841949,0.995226,0.912194,False
7,Logistique (solver = sag),0.841597,0.842049,0.999363,0.913986,False
8,5 plus proches voisins,0.842133,0.842133,1.000000,0.914302,NaN
9,11 plus proches voisins,0.842133,0.842133,1.000000,0.914302,NaN
